In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.6 MB/s eta 0:00:00


In [3]:
import tweepy
from textblob import TextBlob
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch

In [4]:


# Twitter API credentials
consumer_key = "RyxMdVO3V3qOsoYIojmOLgxO0"
consumer_secret = "EYuZwK1HVwIfLmfjGOvg0D3cXQJSn2wNOZ3De2DcPkHxbs0fus"
access_token = "2923530190-94zhelffuRutjVWyLYUfbikzdxkFbHdNgBsKSCK"
access_token_secret = "56cUgA7Kggs93upIhejKQdpSk4zznMKPcSWPVW36Xlkpg"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth)

# Search for tweets about the Warriors
tweets = api.search(q="Ukraine", lang="en", count=100)



# Sentimental tweets using **analysis.sentiment.polarity**

In [5]:
# Analyze the sentiment of each tweet  using Text Blob
positive_tweet = 0
negative_tweet = 0
neutral_tweet = 0

for tweet in tweets:
    analysis = TextBlob(tweet.text)
    if analysis.sentiment.polarity > 0:
        positive_tweet += 1
    elif analysis.sentiment.polarity == 0:
        neutral_tweet += 1
    else:
        negative_tweet += 1
print("Positive tweets: {}".format(positive_tweet))
print("Neutral tweets: {}".format(neutral_tweet))
print("Negative tweets: {}".format(negative_tweet))

Positive tweets: 35
Neutral tweets: 36
Negative tweets: 29


# Sentimental tweets using **Transformers**

In [87]:
# Toekn for transforming the data for the model

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Create the data

raw = []
for tweet in tweets:
  raw.append(str(TextBlob(tweet.text)))

inputs = tokenizer(raw, padding=True, truncation=True, return_tensors="pt")

# Create the Model

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [88]:
# Predictions

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

positive_tweets = 0
negative_tweets = 0
neutral_tweets = 0
for i in range(len(predictions)):
  if predictions[i][1]>0.7:
    positive_tweets += 1
  elif predictions[i][1]<0.3:
    negative_tweets +=1
  else:
    neutral_tweets +=1
    
print("Positive tweets: {}".format(positive_tweets))
print("Neutral tweets: {}".format(neutral_tweets))
print("Negative tweets: {}".format(negative_tweets))

Positive tweets: 18
Neutral tweets: 5
Negative tweets: 77
